<a href="https://colab.research.google.com/github/sarraj-sareef/metaphor-detection-tamil/blob/main/extraction_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Transformers installation
! pip install accelerate -U

# To install from source instead of the last release, comment the command above and uncomment the following one.
# ! pip install git+https://github.com/huggingface/transformers.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 6.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
! pip install transformers datasets sacrebleu evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 46.3 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [ ]:
import pandas as pd
import numpy as np
from datasets import Dataset

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df=pd.read_csv('/content/drive/MyDrive/new extraction dataset - Sheet1.csv')
df.head()

,Name,Metaphor,Source Word,Target word,Target word.1
0,எவனோ ஒருவன் வாசிக்கிறான்,தவம்போல் இருந்து யோசிக்கிறேன்,தவம்,யோசிக்கிறேன்,யோசிக்கிறேன்
1,எவனோ ஒருவன் வாசிக்கிறான்,அந்தக் குழலைபோல் அழுவதற்கு அத்தனை கண்கள் எனக்க...,குழலை,அழுவதற்கு,NaN
2,என்ன சொல்லப் போகிறாய்,இதயம் ஒரு கண்ணாடி,கண்ணாடி,இதயம்,இதயம்
3,ஸ்மையை,மேக்னெட் விழியால்,மேக்னெட்,விழியால்,விழியால்
4,ஸ்மையை,பூவானேன்,பூ,NaN,NaN


In [ ]:
df.drop(['Name','Target word', 'Target word.1'], axis=1, inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split


In [ ]:
# Splitting the data into training (80%) and temp (20%)
train_data, temp_data = train_test_split(df, test_size=0.2, random_state=42)

# Splitting the temp data into validation (50%) and testing (50%)
validation_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

print(f'The training dataset has {len(train_data)} records.')
print(f'The validation dataset has {len(validation_data)} records.')
print(f'The testing dataset has {len(test_data)} records.')


The training dataset has 612 records.
The validation dataset has 77 records.
The testing dataset has 77 records.


In [ ]:
# train_data.reset_index(drop=True, inplace=True)
# test_data.reset_index(drop=True, inplace=True)
hg_train_data = Dataset.from_pandas(train_data)
hg_test_data = Dataset.from_pandas(test_data)
hg_validation_data = Dataset.from_pandas(validation_data)
print(hg_train_data)

Dataset({
    features: ['Metaphor', 'Source Word', '__index_level_0__'],
    num_rows: 612
})


In [ ]:
from datasets import DatasetDict

combined_dataset = DatasetDict({
    'train': hg_train_data,
    'test': hg_test_data,
    'validation' : hg_validation_data
})

# Print the combined dataset information
print(combined_dataset)


DatasetDict({
    train: Dataset({
        features: ['Metaphor', 'Source Word', '__index_level_0__'],
        num_rows: 612
    })
    test: Dataset({
        features: ['Metaphor', 'Source Word', '__index_level_0__'],
        num_rows: 77
    })
    validation: Dataset({
        features: ['Metaphor', 'Source Word', '__index_level_0__'],
        num_rows: 77
    })
})


In [ ]:
print(combined_dataset)

DatasetDict({
    train: Dataset({
        features: ['Metaphor', 'Source Word', '__index_level_0__'],
        num_rows: 612
    })
    test: Dataset({
        features: ['Metaphor', 'Source Word', '__index_level_0__'],
        num_rows: 77
    })
    validation: Dataset({
        features: ['Metaphor', 'Source Word', '__index_level_0__'],
        num_rows: 77
    })
})


In [ ]:
from transformers import AutoTokenizer

# checkpoint = "Hemanth-thunder/english-tamil-mt"
# checkpoint = "mervinpraison/tamil-large-language-model-7b-v1.0"
# checkpoint = "t5-small"
checkpoint = "facebook/mbart-large-50-many-to-many-mmt"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

In [ ]:
# # source_lang = "en"
# # target_lang = "fr"
# prefix = "extract source word: "
# # prefix = "சொற்றொடரைப் பிரித்தெடுக்கவும்: "

# def preprocess_function(examples):
#     inputs = [prefix + example for example in examples["Metaphor"]]
#     targets = [example for example in examples["Source Word"]]
#     model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
#     return model_inputs

In [ ]:
# prefix = "extract source word: "
prefix = "சொற்றொடரைப் பிரித்தெடுக்கவும்: "
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["Metaphor"]]
    model_inputs = tokenizer(inputs, max_length=32, truncation=True)

    labels = []
    for i, target in enumerate(examples["Source Word"]):
        if target is None:  # Check for missing values
            print(f"Found None value in row {i}: {examples['Metaphor'][i]}")
            labels.append("")  # Handle missing values appropriately
        else:
            labels.append(target)

    labels = tokenizer(text=labels, max_length=16, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [ ]:
combined_dataset['train'][3]

{'Metaphor': 'ஒரு தேவதை பார்க்கும் நேரம் இது',
 'Source Word': 'தேவதை',
 '__index_level_0__': 338}

In [ ]:
tokenized_dta = combined_dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/612 [00:00<?, ? examples/s]

Map:   0%|          | 0/77 [00:00<?, ? examples/s]

Map:   0%|          | 0/77 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [ ]:
!pip install evaluate rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=551b28c23386fa57713a830612c6c0a4156d2313188144d490429cae5a878904
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
import evaluate

rouge = evaluate.load("rouge")

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./my_new_extractor_model/",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True,
    fp16=False,
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dta["train"],
    eval_dataset=tokenized_dta["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,0.427916,0.000000,0.000000,0.000000,0.000000,5.844200
2,No log,0.442701,0.000000,0.000000,0.000000,0.000000,5.753200
3,No log,0.426083,0.000000,0.000000,0.000000,0.000000,5.727300
4,No log,0.429427,0.000000,0.000000,0.000000,0.000000,6.051900


TrainOutput(global_step=156, training_loss=0.4237354474189954, metrics={'train_runtime': 113.5823, 'train_samples_per_second': 21.553, 'train_steps_per_second': 1.373, 'total_flos': 158826909007872.0, 'train_loss': 0.4237354474189954, 'epoch': 4.0})

In [ ]:
tokenizer.push_to_hub("sarraj19/my_new_extractor_model")
trainer.push_to_hub()

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5, 'forced_eos_token_id': 2}


events.out.tfevents.1716046470.72a8053604a5.891.1:   0%|          | 0.00/8.03k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/sarraj19/my_new_extractor_model/commit/ab9b1bd924c3ed4ea3d901201e04e20d94134c6b', commit_message='End of training', commit_description='', oid='ab9b1bd924c3ed4ea3d901201e04e20d94134c6b', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
text = "அவன் அருகினிலே\n கணல் மேல் பனிதுளி ஆனேன்"

In [ ]:
from transformers import pipeline

extractor = pipeline("text2text-generation", model="sarraj19/my_new_extractor_model")
# translator(text)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/226 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/10.9k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/992 [00:00<?, ?B/s]

In [ ]:
extractor('கணல் மேல் பனிதுளி ஆனேன்')



[{'generated_text': 'பனிதுளி'}]

In [ ]:
metaphor_values = combined_dataset["validation"]["Metaphor"]
generated_text_list = []
reference_text_list = combined_dataset["validation"]["Source Word"]
print(reference_text_list)


# Print or use the metaphor values
for value in metaphor_values:
    generated_text = translator(value)
    generated_text_list.append(generated_text[0]['generated_text'])
    # print(value, translator(value))
print(generated_text_list)


['பூ', 'பூ', 'அக்மார்க்கு தங்கம்', 'போர் கப்பல்', 'இலை', 'தாவணி', 'ஆகாயம் பூக்கள் தூவும்', 'ஹைகூவே', 'கணல் மேல் பனிதுளி', 'செந்தேன் குழல்', 'மின்னலே', 'கண்மணியே', 'கண்ணே', 'பொன் சிலை', 'வேர்வையாக', 'மின்மினி', 'பெருமையின் எல்லை', 'அணையா விளக்கே', 'ஒலி வட்டம்', 'அடை மழை', 'செப்பு செலை', 'சதுக்கம்', 'கடலுக்குள்', 'ஊர் கும்பிடும் குல சாமி', 'வானம்', 'சஹாரா பூக்கள் பூத்ததோ', 'மகரந்த காட்டின் மான்குட்டியின்', 'அக்கினி பழமுன்னு', 'தங்கமணி முத்துமணி', 'பால்மழை', 'மௌனங்கள்', 'அஞ்சு வயதுப்பிள்ளை', 'தேன்களின் இருக்கை', 'பூ', 'வாசனை இருக்கும் வரையினில் சிரிக்கும் பூக்களின் கதை', 'வான் நிலா தரும் ஒளி', 'தங்கமே', 'கரும்பாறை', 'வான்', 'பூ', 'தேவதை', 'சூரிய பந்தொன்று', 'பிள்ளை', 'நிலவு', 'காத்தாடி', 'நதி', 'கண்மணி', 'நூலில் ஆடும் மழையாகி', 'பாதி', 'உலக அழகியே', 'பொன்னம்மா', 'கருபட்டி', 'பொன்', 'தீ', 'முத்தே என் முத்தாரமே', 'எரிமலை', 'தாய்ப்பால் மழை', 'காந்தம்', 'பொல்லாத என் இதயம்', 'நீரும் செம்புல சேறும் கலந்தது', 'கண் கட்டுதே', 'சூரியன் , வான்முகில்', 'பொன் வீணை', 'பேனாவில் ஊற்றி வைத்தது', 'சூரியன் 

In [ ]:
generated_text_list = ['பூ', 'மலரும்', 'அக்மார்க்கு தங்கம்', 'போர் கப்பல்', 'நீ இலை', 'தாவணி', 'பூக்கள் தூவும்', 'கை கால்கள்', 'பனிதுளி', 'செந்தேன் குழல்', 'மின்னலே', 'கண்மணி', 'கண்ணான', 'பொன் சிலை', 'பொல்லாத போர்', 'மின்', 'எல்லை', 'பாட்டும் அணை', 'ஒலி வட்டம்', 'மழை', 'செப்பு செலை', 'சதுக்கம்', 'கடலுக்குள்', 'குல சாமி', 'வானம்', 'சஹாரா', 'மான்குட்டி', 'பழமுன்னு', 'தங்கமணி முத்துமணி', 'பால்மழை', 'அவளது மொழி', 'அஞ்சு வயதுள்ளை', 'உதடு தேன்களின் இருக்கை', ' பூவே', 'பூக்களின் கதை', 'நிலா தரும் ஒளி', 'பேபி தங்கமே', 'கரும்பாறை', 'பூமாலை', 'பூ', 'தேவதை', 'சூரிய பந்தொன்று', 'கண்மணி', 'நிலவு', 'காத்தாடி', 'Harmon நதி', 'கண்மணி', 'நூலில் ஆடும் மழை', 'உந்தன் மீதி', 'அழகியே', 'பொன்னம்', 'கருப்பட்டி', 'பொன்மேனி', 'தீயே', 'முத்தாரமே', 'விழி எரிமலை', 'மழை', 'அன்பு', 'நில்லாத என் உயிரோ', 'செம்புல சேறும்', 'காதல் கண் கட்டுதே', 'சூரியன்', 'பொன் வீணை', 'பேனா', 'சூரியன்', 'எழுத்து பிழை', 'தேசிய கீதம்', 'மெல்லிசை', 'ஸ்வாசமே', 'புன்னகை வெயில்', 'பூக்காடு', 'குறிஞ்சிப் பூ', 'நிலவுங்க எல்லாம் இங்க தேவை', 'நிலம் நாலு பக்கம்', 'கொல்லுதே', 'இருள்', 'சாபம்']
reference_text_list = combined_dataset["validation"]["Source Word"]

In [ ]:
# # Initialize a list to store the indices of None values
# none_indices = []

# # Iterate through the list and find indices of None values
# for i, text in enumerate(reference_text_list):
#     if text is None:
#         none_indices.append(i)

# # Print the indices of None values
# print("Indices of None values:", none_indices, reference_text_list[none_indices[0]],combined_dataset["validation"][none_indices])


In [ ]:
!pip install rouge_score

In [ ]:
def calculate_rouge_n(reference_texts, generated_texts, n=1):
    total_precision = 0
    total_recall = 0
    total_f1_score = 0
    valid_samples = 0

    for reference, generated in zip(reference_texts, generated_texts):
        # Skip the pair if either is None
        if reference is None or generated is None:
            continue

        # Tokenize the sentences into words
        ref_words = reference.split()
        gen_words = generated.split()

        # Create n-grams
        ref_ngrams = set(zip(*[ref_words[i:] for i in range(n)]))
        gen_ngrams = set(zip(*[gen_words[i:] for i in range(n)]))

        # Calculate overlap
        overlap = len(ref_ngrams & gen_ngrams)
        ref_count = len(ref_ngrams)
        gen_count = len(gen_ngrams)

        # Calculate precision, recall, and F1 score for this pair
        precision = overlap / gen_count if gen_count > 0 else 0
        recall = overlap / ref_count if ref_count > 0 else 0
        f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

        # Add to total scores
        total_precision += precision
        total_recall += recall
        total_f1_score += f1_score
        valid_samples += 1

    # Calculate average scores
    avg_precision = total_precision / valid_samples if valid_samples > 0 else 0
    avg_recall = total_recall / valid_samples if valid_samples > 0 else 0
    avg_f1_score = total_f1_score / valid_samples if valid_samples > 0 else 0

    return avg_precision, avg_recall, avg_f1_score

# Example usage:
# reference_texts = ["கண்ணாடி", "நெருப்ப"]
# generated_texts = ["கண்ணாடி", "நெருப்பு "]

# Calculate average ROUGE-1
avg_precision_1, avg_recall_1, avg_f1_score_1 = calculate_rouge_n(reference_text_list, generated_text_list, n=1)
print("Average ROUGE-1 Precision:", avg_precision_1)
print("Average ROUGE-1 Recall:", avg_recall_1)
print("Average ROUGE-1 F1 Score:", avg_f1_score_1)


In [ ]:
import nltk
total_score = 0
for i in range(len(reference_text_list)):
  x = min(len(reference_text_list[i]),len(generated_text_list[i]))
  score = (nltk.translate.chrf_score.sentence_chrf(reference_text_list[i], generated_text_list[i], min_len=1, max_len=x-1, beta=3.0, ignore_whitespace=True))
  total_score+=score
  print(reference_text_list[i], generated_text_list[i], score)
avg_score = total_score/len(reference_text_list)
print(avg_score)

In [ ]:
import nltk
from nltk.translate.bleu_score import sentence_bleu

def calculate_character_bleu(candidate, reference):
    # Tokenize the strings into character-level n-grams (e.g., bigrams)
    candidate_tokens = list(candidate)
    reference_tokens = list(reference)

    # Compute BLEU score (using 2-grams, you can adjust n as needed)
    bleu_score = sentence_bleu([reference_tokens], candidate_tokens, weights=(0.5, 0.5))

    return bleu_score

# Example usage
total_score = 0
for i in range(len(reference_text_list)):
  # x = min(len(reference_text_list[i]),len(generated_text_list[i]))
  score = calculate_character_bleu(reference_text_list[i], generated_text_list[i])
  total_score+=score
  print(reference_text_list[i], generated_text_list[i], score)
avg_score = total_score/len(reference_text_list)
print(avg_score)

print(f"Character BLEU Score: {avg_score:.4f}")


In [ ]:
! pip install levenshtein
from Levenshtein import ratio

In [ ]:
total_score = 0
for i in range(len(reference_text_list)):
  # x = min(len(reference_text_list[i]),len(generated_text_list[i]))
  score = ratio(reference_text_list[i], generated_text_list[i])
  total_score+=score
  print(reference_text_list[i], generated_text_list[i], score)
avg_score = total_score/len(reference_text_list)
print(avg_score)
